In [6]:
import fitz  # PyMuPDF library for PDF processing
import re
from collections import Counter
import os

# --- Configuration: Define skill sets for analysis ---

# CORE_SKILLS are a broad set of skills the tool will search for in the resume.
# These can be technical, soft skills, tools, methodologies, etc.
CORE_SKILLS = [
    "Python", "SQL", "Machine Learning", "Data Analysis", "Deep Learning",
    "NLP", "Cloud", "AWS", "Azure", "GCP", "Docker", "Kubernetes",
    "Git", "GitHub", "Jupyter", "Pandas", "NumPy", "Scikit-learn",
    "TensorFlow", "PyTorch", "Tableau", "Power BI", "Excel", "Spark",
    "R", "Java", "C++", "JavaScript", "HTML", "CSS", "API", "REST",
    "Agile", "Scrum", "Project Management", "Communication", "Problem Solving",
    "Critical Thinking", "Leadership", "Teamwork", "Problem-solving", "Collaboration"
]

# TARGET_SKILLS are a subset of highly desired skills for a hypothetical job role.
# The tool will use this list to identify 'keyword gaps' in the resume.
TARGET_SKILLS = [
    "Python", "SQL", "Machine Learning", "Deep Learning", "TensorFlow",
    "AWS", "Docker", "Kubernetes", "Git", "API", "Scikit-learn", "NLP",
    "Data Analysis", "Cloud", "Problem Solving" # Added some common ones for better demo
]


# --- Function 1: Extract Text from PDF ---
def extract_text_from_pdf(pdf_path):
    """
    Extracts text content from a specified PDF file.

    Args:
        pdf_path (str): The absolute or relative path to the PDF file.

    Returns:
        str: The concatenated text content from all pages of the PDF.

    Raises:
        FileNotFoundError: If the specified PDF file does not exist.
        Exception: For other errors encountered during PDF processing (e.g., corrupted file).
    """
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"Error: PDF file not found at '{pdf_path}'")

    text_content = ""
    try:
        # Open the PDF document using PyMuPDF (fitz)
        document = fitz.open(pdf_path)
        for page_num in range(len(document)):
            # Load each page and extract its text
            page = document.load_page(page_num)
            text_content += page.get_text()
        document.close() # Always close the document after processing
    except Exception as e:
        # Catch generic exceptions during PDF parsing, indicating a potential issue
        # with the PDF file itself or PyMuPDF installation.
        raise Exception(f"Error extracting text from PDF '{pdf_path}': {e}")
    return text_content


# --- Function 2: Count Skill Mentions ---
def count_skill_mentions(text, skills_list):
    """
    Counts the occurrences of each skill from a given list within the provided text.
    Matching is case-insensitive and uses word boundaries to avoid partial matches
    (e.g., 'Python' matches 'Python' but not 'Pythonic' without more advanced regex).

    Args:
        text (str): The complete text extracted from the resume.
        skills_list (list): A list of strings, where each string is a skill to count.

    Returns:
        dict: A dictionary where keys are the original skill names (case-preserved from skills_list)
              and values are their respective integer counts.
    """
    skill_counts = Counter()
    # Convert the entire text to lowercase once for efficient case-insensitive matching
    text_lower = text.lower()

    for skill in skills_list:
        # Construct a regular expression pattern for each skill:
        # r'\b' ensures word boundaries (e.g., 'ml' won't match 'email').
        # re.escape(skill.lower()) escapes any special regex characters in the skill name
        # and converts the skill name to lowercase for matching against text_lower.
        pattern = r'\b' + re.escape(skill.lower()) + r'\b'
        # Find all non-overlapping matches of the pattern in the lowercase text
        matches = re.findall(pattern, text_lower)
        # Store the count using the original skill name from the skills_list
        skill_counts[skill] = len(matches)

    return dict(skill_counts)


# --- Function 3: Suggest Improvements Based on Keyword Gaps ---
def suggest_improvements(resume_skills_counts, target_skills, min_mention_threshold=1):
    """
    Generates suggestions for resume improvement by identifying keyword gaps.
    It compares skills found in the resume against a list of desired target skills.

    Args:
        resume_skills_counts (dict): A dictionary where keys are skills found in the resume
                                     and values are their counts.
        target_skills (list): A list of highly desired skills (e.g., from a job description).
        min_mention_threshold (int): The minimum number of times a skill must be mentioned
                                     in the resume to be considered "adequately present."

    Returns:
        list: A list of strings, each representing an actionable improvement suggestion.
    """
    suggestions = []

    # Create a set of skills found in the resume (converted to lowercase for comparison)
    # Only include skills that meet or exceed the minimum mention threshold.
    found_skills_adequately_present = {
        skill.lower() for skill, count in resume_skills_counts.items()
        if count >= min_mention_threshold
    }

    # Identify target skills that are completely missing from the resume
    missing_target_skills = []
    for target_skill in target_skills:
        if target_skill.lower() not in found_skills_adequately_present:
            missing_target_skills.append(target_skill)

    if missing_target_skills:
        suggestions.append(
            f"Consider adding or emphasizing the following target skills: "
            f"{', '.join(missing_target_skills)}. These were either not found "
            f"or had insufficient mentions (below {min_mention_threshold})."
        )

    # Identify target skills that are present but have low mentions (below threshold)
    low_mention_target_skills = []
    for skill in target_skills:
        # Check if the skill was found but its count is below the threshold
        if skill.lower() in resume_skills_counts and resume_skills_counts[skill] < min_mention_threshold:
            low_mention_target_skills.append(f"{skill} ({resume_skills_counts[skill]} mention{'s' if resume_skills_counts[skill] != 1 else ''})")

    if low_mention_target_skills:
        suggestions.append(
            f"The following target skills are present in your resume but could be "
            f"highlighted more, as their mentions are low (below {min_mention_threshold}): "
            f"{', '.join(low_mention_target_skills)}. Consider elaborating on projects "
            f"or experiences where these skills were applied."
        )

    if not suggestions:
        suggestions.append("Your resume appears to align well with the target skills. Keep up the great work!")

    return suggestions


# --- Main Execution Logic ---
def main(resume_path, min_skill_mentions=1, target_skills_file=None):
    """
    Main function for the Resume Analyzer.
    Takes the PDF resume path, optional minimum skill mentions,
    and optional target skills file as input.
    Orchestrates PDF text extraction, skill counting,
    and improvement suggestion generation, then prints results.
    """

    # Determine which set of target skills to use (custom file or default)
    current_target_skills = TARGET_SKILLS # Start with default

    if target_skills_file:
        try:
            # Attempt to read skills from the specified file
            with open(target_skills_file, 'r', encoding='utf-8') as f:
                # Read each non-empty line and strip whitespace
                file_skills = [line.strip() for line in f if line.strip()]
            if file_skills:
                current_target_skills = file_skills
                print(f"Using custom target skills loaded from: '{target_skills_file}'")
            else:
                print(f"Warning: Custom target skills file '{target_skills_file}' is empty. Using default TARGET_SKILLS.")
        except FileNotFoundError:
            print(f"Warning: Custom target skills file not found at '{target_skills_file}'. Using default TARGET_SKILLS.")
        except Exception as e:
            print(f"Error reading custom target skills file '{target_skills_file}': {e}. Using default TARGET_SKILLS.")

    print(f"\n--- RESUME ANALYSIS REPORT for: {os.path.basename(resume_path)} ---")

    try:
        # Step 1: Extract Text
        print("\n--- 1. Extracting Text from PDF ---")
        resume_text = extract_text_from_pdf(resume_path)
        # Optional: Print a snippet of extracted text for verification/debugging
        # print(f"Extracted {len(resume_text)} characters. First 200 chars:\n{resume_text[:200]}...")
        print("Text extraction complete.")


        # Step 2: Count Key Skill Mentions
        print("\n--- 2. Counting Key Skill Mentions ---")
        found_skills_counts = count_skill_mentions(resume_text, CORE_SKILLS)

        # Filter and sort skills that were actually found with a count > 0
        present_skills_sorted = sorted(
            [(skill, count) for skill, count in found_skills_counts.items() if count > 0],
            key=lambda item: item[1], # Sort by count
            reverse=True # Descending order
        )

        if present_skills_sorted:
            print("Detected skills and their frequencies:")
            for skill, count in present_skills_sorted:
                print(f"  - {skill}: {count} mention{'s' if count != 1 else ''}")
        else:
            print("No predefined CORE_SKILLS were detected in the resume text.")


        # Step 3: Suggest Improvements based on Keyword Gaps
        print("\n--- 3. Improvement Suggestions ---")
        print(f"Analyzing against target skills (threshold for 'present' = {min_skill_mentions} mention(s)):")
        print(f"Target Skills: {', '.join(current_target_skills)}")
        suggestions = suggest_improvements(found_skills_counts, current_target_skills, min_skill_mentions)
        for i, suggestion_text in enumerate(suggestions):
            print(f"  {i+1}. {suggestion_text}")

    except FileNotFoundError as e:
        print(f"\nError: {e}")
        print("Please double-check the path to your PDF resume file.")
    except Exception as e:
        print(f"\nAn unexpected error occurred during analysis: {e}")
        print("Ensure PyMuPDF is correctly installed (`pip install PyMuPDF`) and your PDF is not corrupted or image-based.")

# Define the path to your resume PDF here
# Replace '/path/to/your/resume.pdf' with the actual path to your file
resume_file_path = '/content/my_resume.pdf'

# Define optional parameters (set to None to use defaults)
min_mentions = 1 # Example: Set to 2 to require at least 2 mentions
custom_target_skills_file_path = None # Example: Set to '/path/to/your/target_skills.txt'

# Call the main function with the defined parameters
main(resume_file_path, min_mentions, custom_target_skills_file_path)


--- RESUME ANALYSIS REPORT for: my_resume.pdf ---

--- 1. Extracting Text from PDF ---
Text extraction complete.

--- 2. Counting Key Skill Mentions ---
Detected skills and their frequencies:
  - Machine Learning: 5 mentions
  - Python: 2 mentions
  - Cloud: 2 mentions
  - Jupyter: 2 mentions
  - Pandas: 2 mentions
  - Scikit-learn: 2 mentions
  - CSS: 2 mentions
  - API: 2 mentions
  - SQL: 1 mention
  - Deep Learning: 1 mention
  - NLP: 1 mention
  - Azure: 1 mention
  - Git: 1 mention
  - GitHub: 1 mention
  - NumPy: 1 mention
  - TensorFlow: 1 mention
  - Power BI: 1 mention
  - R: 1 mention
  - Java: 1 mention
  - JavaScript: 1 mention
  - HTML: 1 mention
  - REST: 1 mention
  - Agile: 1 mention

--- 3. Improvement Suggestions ---
Analyzing against target skills (threshold for 'present' = 1 mention(s)):
Target Skills: Python, SQL, Machine Learning, Deep Learning, TensorFlow, AWS, Docker, Kubernetes, Git, API, Scikit-learn, NLP, Data Analysis, Cloud, Problem Solving
  1. Consider 